## Задание

Мы будем работать с (частичными) данными lenta.ru отсюда: https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/ Ниже примерный список того, что можно попробовать в ходе решения.

1. Проведите предобработку текста: уберите небуквенные символы, проведите токенизацию, лемматизируйте текст. Разбейте данные на train и test для задачи классификации (в качестве метки класса будем использовать поле topic_label).В качестве данных для классификации в пунктах 3 и 5 попробуйте взять
    - только заголовки (title)
    - только тексты новости (text)
    - и то, и другое
2. Обучите fastText для классификации текстов по темам (baseline).
3. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста / заголовка / текста+заголовка как среднего вектора входящих в него слов. 
     - (Бонус) Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.
4. Обучите на полученных средних векторах алгоритм классификации (логистическая регрессия, SVM, KNN, однослойную полносвязную нейросеть). 
5. Обучите сверточную сеть, на вход можно подавать предобученные эмбеддинги из п. 4 **(для слов, не для предложений)**.

In [0]:
! wget -O lenta-ru-train.csv https://www.dropbox.com/s/kdupcw1llbdbqwl/lenta-ru-train.csv?dl=0

--2020-02-19 19:32:05--  https://www.dropbox.com/s/kdupcw1llbdbqwl/lenta-ru-train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kdupcw1llbdbqwl/lenta-ru-train.csv [following]
--2020-02-19 19:32:06--  https://www.dropbox.com/s/raw/kdupcw1llbdbqwl/lenta-ru-train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc717326034485f397c1843c4e4a.dl.dropboxusercontent.com/cd/0/inline/AyaD-4iISIoaCweUHIdlyBf97S1AskhmHzC_fsHO6Dgv8Sj9KPRagFx_N_qgyQhEQ3wzDVLEzur5DkVDYorGPhP4P9wDZQ5CrIzad3Eu30cpMWKIjf5_pr3f3ZGDa5sC41s/file# [following]
--2020-02-19 19:32:06--  https://uc717326034485f397c1843c4e4a.dl.dropboxusercontent.com/cd/0/inline/AyaD-4iISIoaCweUHIdlyBf97S1AskhmHzC_fsHO6Dgv8Sj9KPRagFx_N_qgyQhEQ3wzDVLEzur5DkVDYorGPhP4P9w

In [0]:
import pandas as pd

lenta = pd.read_csv('lenta-ru-train.csv')
lenta.head()

,title,text,topic,topic_label
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1


In [0]:
lenta.topic.value_counts()

Экономика          63693
Спорт              51396
Культура           43077
Наука и техника    42507
Бизнес              5964
Name: topic, dtype: int64

In [0]:
len(lenta)

206637

Ниже пример, как можно сделать файл с решением для Kaggle.

In [0]:
! wget -O lenta-ru-test.csv https://www.dropbox.com/s/9av90tgqob1slnj/lenta-ru-test.csv?dl=0

In [0]:
from random import randint

topics = ['Экономика', 'Спорт', 'Культура', 'Наука и техника', 'Бизнес']
test_data = pd.read_csv('lenta-ru-test.csv')
test_data.head()

,title,text
0,ВТБ усовершенствует систему корпоративного упр...,ВТБ продолжает работу по совершенствованию сис...
1,Более половины французов сочли вероятным дефол...,"53 процента французов боятся, что их правитель..."
2,Лев MGM оставил отпечатки лап на бульваре Голл...,"Лев Лео, символ студии Metro-Goldwyn-Mayer, ст..."
3,Адам Сэндлер и Кристина Ричи займутся порно,Адам Сэндлер и Кристина Ричи сыграют в комедии...
4,ФИФА заплатила создательнице волка Забиваки 50...,Международная федерация футбола (ФИФА) выкупил...


In [0]:
test_data['label'] = [topics[randint(0,4)] for i in range(len(test_data)) ]
test_data.head()

,title,text,label
0,ВТБ усовершенствует систему корпоративного упр...,ВТБ продолжает работу по совершенствованию сис...,Бизнес
1,Более половины французов сочли вероятным дефол...,"53 процента французов боятся, что их правитель...",Наука и техника
2,Лев MGM оставил отпечатки лап на бульваре Голл...,"Лев Лео, символ студии Metro-Goldwyn-Mayer, ст...",Спорт
3,Адам Сэндлер и Кристина Ричи займутся порно,Адам Сэндлер и Кристина Ричи сыграют в комедии...,Наука и техника
4,ФИФА заплатила создательнице волка Забиваки 50...,Международная федерация футбола (ФИФА) выкупил...,Наука и техника


In [0]:
test_data['label'].to_csv('submission.csv', header = None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
